# Notebook 1: Data Collection and Inventory

Purpose: download and organize all 15 datasets required for the analysis.



Datasets to collect:

1. Wind Speed (Global Wind Atlas)

2. Bathymetry (GEBCO 2020)

3. Power Grid (World Bank/OSM)

4. Ports (Morocco ANP)

5. Shoreline (GSHHG)

6. Tourism POIs (OpenStreetMap)

7. Airports (OpenStreetMap)

8. Sediment Thickness (NOAA)

9. Submarine Cables (TeleGeography)

10. Shipping Routes (EMODnet)

11. Protected Areas (UNEP-WCMC)

12. EEZ Boundaries (Marine Regions)

13. Blue Flag Beaches (Blue Flag Global)

14. Bird Migration Routes (Movebank)

15. Offshore Wind Potential (World Bank/ESMAP)



Expected duration: 2-3 hours (download speed dependent).

## Colab Quick Start
If you open this notebook in Google Colab:

1. Mount or clone the repo so `morocco-offshore-wind-gis` lives under `/content`.
2. Run the next cell once to install requirements and pull latest.

Skip this cell when running locally.

In [1]:
# Local bootstrap only (no Colab)
from pathlib import Path
import os
import sys
import subprocess

PROJECT_ROOT = Path("C:/Users/NameLess/Desktop/morocco-offshore-wind-gis").resolve()
os.chdir(PROJECT_ROOT)
REQ_FILE = PROJECT_ROOT / "requirements.txt"

print(f"Working directory: {PROJECT_ROOT}")
if REQ_FILE.exists():
    print("requirements.txt found. If not already done: activate your venv then run pip install -r requirements.txt")
else:
    print("requirements.txt missing; confirm you are in the project root.")

Working directory: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis
requirements.txt found. If not already done: activate your venv then run pip install -r requirements.txt


## Setup and Imports

In [2]:
# Setup paths and imports (local)
from pathlib import Path
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import rasterio
import xarray as xr

PROJECT_ROOT = Path("C:/Users/NameLess/Desktop/morocco-offshore-wind-gis").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
OUTPUTS_DIR = DATA_DIR / "outputs"

for p in [DATA_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"RAW_DATA_DIR: {RAW_DATA_DIR}")

PROJECT_ROOT: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis
RAW_DATA_DIR: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw


## Data Inventory Checklist

In [30]:
# Dataset catalog

DATASETS = {

    "Wind Speed": {

        "file": "wind_speed_100m.tif",

        "alt_files": [],

        "source": "Global Wind Atlas 3.0",

        "url": "https://globalwindatlas.info/",

        "format": "GeoTIFF",

        "manual_download": True

    },

    "Bathymetry": {

        "file": "bathymetry_gebco.nc",

        "alt_files": ["bathymetry_gebco.tif", "bathymetry_morocco_etopo2022.tiff"],

        "source": "GEBCO 2020 (primary) / NOAA ETOPO fallback",

        "url": "https://www.gebco.net/data_and_products/gridded_bathymetry_data/",

        "format": "NetCDF",

        "manual_download": True

    },

    "Power Grid": {

        "file": "power_grid_morocco.geojson",

        "alt_files": ["power_grid.geojson", "power_grid.shp"],

        "source": "World Bank (provided) / OSM fallback",

        "url": "https://energydata.info/",

        "format": "GeoJSON",

        "manual_download": False

    },

    "Ports": {

        "file": "ports_morocco.geojson",

        "alt_files": ["ports.geojson", "ports.csv"],

        "source": "OpenStreetMap (Geofabrik)",

        "url": "https://download.geofabrik.de/africa/morocco.html",

        "format": "GeoJSON",

        "manual_download": False

    },

    "Shoreline": {

        "file": "shoreline_morocco.shp",

        "alt_files": [
            "shoreline_morocco.gpkg",
            "morocco_geofabrik/gis_osm_coastlines_free_1.shp",
            "ne_10m_coastline.shp"
        ],

        "source": "GSHHG v2.3.7 (target) / OSM coastline fallback",

        "url": "https://www.soest.hawaii.edu/pwessel/gshhg/",

        "format": "Shapefile",

        "manual_download": True

    },

    "Tourism": {

        "file": "tourism_morocco.geojson",

        "alt_files": ["tourism_morocco.shp"],

        "source": "OpenStreetMap (Geofabrik)",

        "url": "https://download.geofabrik.de/africa/morocco.html",

        "format": "GeoJSON",

        "manual_download": False

    },

    "Airports": {

        "file": "airports_morocco.geojson",

        "alt_files": ["airports_morocco.shp"],

        "source": "OpenStreetMap (Geofabrik)",

        "url": "https://download.geofabrik.de/africa/morocco.html",

        "format": "GeoJSON",

        "manual_download": False

    },

    "Sediment Thickness": {

        "file": "sediment_thickness.nc",

        "alt_files": ["sediment_thickness.tif", "sediment_thickness_morocco.tif", "GlobSed-v3.nc"],

        "source": "NOAA Geophysics",

        "url": "https://www.ngdc.noaa.gov/mgg/sedthick/",

        "format": "NetCDF or GeoTIFF",

        "manual_download": True

    },

    "Submarine Cables": {

        "file": "submarine_cables.geojson",

        "alt_files": [],

        "source": "TeleGeography",

        "url": "https://github.com/telegeography/www.submarinecablemap.com",

        "format": "GeoJSON",

        "manual_download": False,

        "direct_url": "https://raw.githubusercontent.com/telegeography/www.submarinecablemap.com/master/web/public/api/v3/cable/cable-geo.json"

    },

    "Shipping Routes": {

        "file": "shipping_density.tif",

        "alt_files": ["vesseldensity_all_2020.tif", "shipping_density_emodnet_2020.tif", "shipping_density_morocco_2020.tif"],

        "source": "EMODnet",

        "url": "https://www.emodnet-humanactivities.eu/view-data.php",

        "format": "GeoTIFF",

        "manual_download": True

    },

    "Protected Areas": {

        "file": "protected_areas_morocco.shp",

        "alt_files": ["protected_areas_morocco.gpkg"],

        "source": "UNEP-WCMC",

        "url": "https://www.protectedplanet.net/en",

        "format": "Shapefile",

        "manual_download": True

    },

    "EEZ": {

        "file": "morocco_eez.shp",

        "alt_files": ["morocco_eez.gpkg"],

        "source": "Marine Regions",

        "url": "https://www.marineregions.org/downloads.php",

        "format": "Shapefile",

        "manual_download": True

    },

    "Blue Flag Beaches": {

        "file": "blue_flag_beaches.csv",

        "alt_files": [],

        "source": "Blue Flag Global",

        "url": "https://www.blueflag.global/all-bf-sites",

        "format": "CSV",

        "manual_download": True

    },

    "Bird Migration": {

        "file": "bird_migration_routes.shp",

        "alt_files": ["bird_migration_routes.geojson"],

        "source": "Movebank or BirdLife",

        "url": "https://www.movebank.org/",

        "format": "Shapefile",

        "manual_download": True

    },

    "Offshore Wind Potential": {

        "file": "offshore_wind_potential_worldbank.xlsx",

        "alt_files": [],

        "source": "World Bank ESMAP",

        "url": "https://esmap.org/",

        "format": "Excel",

        "manual_download": True

    }

}



print(f"Total datasets to collect: {len(DATASETS)}")

Total datasets to collect: 15


## Check Current Data Status

In [31]:
def check_data_status():

    """Check which datasets have been downloaded (supports alternate filenames)."""

    status_list = []

    for name, info in DATASETS.items():

        candidates = [info["file"]] + info.get("alt_files", [])

        found_path = None

        for fname in candidates:

            file_path = RAW_DATA_DIR / fname

            if file_path.exists():

                found_path = file_path

                break

        if found_path:

            size_mb = found_path.stat().st_size / (1024 * 1024)

            size_str = f"{size_mb:.2f} MB"

            status_icon = "OK"

            filename_display = found_path.name

        else:

            size_str = "-"

            status_icon = "MISSING"

            filename_display = info["file"]

        status_list.append({

            "Dataset": name,

            "Status": status_icon,

            "Filename": filename_display,

            "Size": size_str,

            "Source": info["source"]

        })

    return pd.DataFrame(status_list)



# Display current status

status_df = check_data_status()

print("Data collection status:\n")

print(status_df.to_string(index=False))

downloaded = (status_df["Status"] == "OK").sum()

total = len(status_df)

progress = (downloaded / total) * 100

print(f"\nProgress: {downloaded}/{total} ({progress:.1f}%) datasets collected")

Data collection status:

                Dataset  Status                               Filename     Size                                         Source
             Wind Speed      OK                    wind_speed_100m.tif 33.16 MB                          Global Wind Atlas 3.0
             Bathymetry      OK      bathymetry_morocco_etopo2022.tiff 52.92 MB     GEBCO 2020 (primary) / NOAA ETOPO fallback
             Power Grid      OK             power_grid_morocco.geojson  0.01 MB           World Bank (provided) / OSM fallback
                  Ports      OK                  ports_morocco.geojson  0.00 MB                      OpenStreetMap (Geofabrik)
              Shoreline      OK                  shoreline_morocco.shp  6.49 MB GSHHG v2.3.7 (target) / OSM coastline fallback
                Tourism      OK                tourism_morocco.geojson  2.10 MB                      OpenStreetMap (Geofabrik)
               Airports      OK               airports_morocco.geojson  0.12 MB       

In [ ]:
# Debug the wind speed file path and directory contents
from pathlib import Path

print(f"Kernel CWD: {Path.cwd()}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"RAW_DATA_DIR exists? {RAW_DATA_DIR.exists()}")

wind_path = RAW_DATA_DIR / "wind_speed_100m.tif"
print(f"Wind path: {wind_path}")
print(f"Wind exists? {wind_path.exists()}")

if wind_path.exists():
    size_mb = wind_path.stat().st_size / 1_048_576
    print(f"Wind size: {size_mb:.2f} MB")
    print("Contents of raw dir:")
    for p in sorted(RAW_DATA_DIR.iterdir()):
        print(f"  - {p.name}")

## Dataset 1: Wind Speed (Global Wind Atlas)



Manual download instructions:

1. Go to https://globalwindatlas.info/

2. Choose Download, Wind Speed at 100 m

3. Region bounds: north 36, south 21, west -20, east -1

4. Format: GeoTIFF

5. Save to data/raw/wind_speed_100m.tif

In [ ]:
# Verify wind speed file

wind_speed_path = RAW_DATA_DIR / "wind_speed_100m.tif"



if wind_speed_path.exists():

    print("Wind speed data found.")

    import rasterio

    with rasterio.open(wind_speed_path) as src:

        print(f"  CRS: {src.crs}")

        print(f"  Shape: {src.shape}")

        print(f"  Bounds: {src.bounds}")

        print(f"  NoData: {src.nodata}")

else:

    print("Wind speed data not found. Please download manually.")

## Dataset 2: Bathymetry (GEBCO 2020)



Manual download instructions:

1. Go to https://www.gebco.net/data_and_products/gridded_bathymetry_data/

2. Select GEBCO 2020 grid

3. Region bounds: north 36, south 21, west -20, east -1

4. Format: NetCDF or GeoTIFF

5. Save to data/raw/bathymetry_gebco.nc

In [25]:
# Verify bathymetry file

bathymetry_path = RAW_DATA_DIR / "bathymetry_gebco.nc"



if bathymetry_path.exists():

    print("Bathymetry data found.")

    import xarray as xr

    ds = xr.open_dataset(bathymetry_path)

    print(f"  Variables: {list(ds.data_vars)}")

    print(f"  Coordinates: {list(ds.coords)}")

    print(f"  Dimensions: {ds.dims}")

    ds.close()

else:

    print("Bathymetry data not found. Please download manually.")

Bathymetry data not found. Please download manually.


## Dataset: Sediment Thickness (GlobSed subset for Morocco EEZ)

In [29]:
from pathlib import Path
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Paths
sediment_nc = RAW_DATA_DIR / "GlobSed-v3.nc"
target_tif = RAW_DATA_DIR / "sediment_thickness_morocco.tif"
target_png = OUTPUTS_DIR / "sediment_thickness_morocco_map.png"

# Coordinate bounds (Morocco EEZ region)
lat_min, lat_max = 21.0, 37.0
lon_min, lon_max = -20.0, -1.0

if not sediment_nc.exists():
    print(f"Sediment dataset not found: {sediment_nc}")
else:
    # Prefer dask-backed lazy read; fall back to in-memory if dask missing
    try:
        import dask  # noqa: F401
        chunk_opts = {}
        print("Using dask-backed lazy loading.")
    except ImportError:
        chunk_opts = None
        print("dask not installed; opening without chunking (will load subset to memory).")

    ds = xr.open_dataset(sediment_nc, chunks=chunk_opts)
    print("Dataset summary (lazy):")
    print(ds)
    print("Variables:", list(ds.data_vars))

    # Identify coord keys
    def coord_key(ds, prefix):
        for k in ds.coords:
            if k.lower().startswith(prefix):
                return k
        raise KeyError(f"No coordinate starting with '{prefix}' found.")

    lat_key = coord_key(ds, "lat")
    lon_key = coord_key(ds, "lon")

    data_var = list(ds.data_vars)[0]
    da = ds[data_var]

    # Build slices respecting coord direction
    lat_vals = da[lat_key].values
    lon_vals = da[lon_key].values
    lat_slice = slice(lat_min, lat_max) if lat_vals[0] < lat_vals[-1] else slice(lat_max, lat_min)
    lon_slice = slice(lon_min, lon_max) if lon_vals[0] < lon_vals[-1] else slice(lon_max, lon_min)

    da_clip = da.sel({lat_key: lat_slice, lon_key: lon_slice})

    # Load clipped data only
    da_clip = da_clip.astype("float32").compute()

    # Units handling
    units = da_clip.attrs.get("units", "unknown")
    data_np = da_clip.values
    if units.lower() in {"km", "kilometer", "kilometers", "kilometres", "kilometre"}:
        data_np = data_np * 1000.0
        units = "m"
    elif units.lower() in {"m", "meter", "meters", "metre", "metres"}:
        units = "m"

    # Spatial transform
    lon_vals_clip = da_clip[lon_key].values
    lat_vals_clip = da_clip[lat_key].values
    resx = float(abs(lon_vals_clip[1] - lon_vals_clip[0])) if lon_vals_clip.size > 1 else 0.01
    resy = float(abs(lat_vals_clip[1] - lat_vals_clip[0])) if lat_vals_clip.size > 1 else 0.01
    transform = from_origin(west=float(lon_vals_clip.min()), north=float(lat_vals_clip.max()), xsize=resx, ysize=resy)

    # Write GeoTIFF
    profile = {
        "driver": "GTiff",
        "height": data_np.shape[0],
        "width": data_np.shape[1],
        "count": 1,
        "dtype": "float32",
        "crs": "EPSG:4326",
        "transform": transform,
        "compress": "lzw",
    }
    target_tif.parent.mkdir(parents=True, exist_ok=True)
    with rasterio.open(target_tif, "w", **profile) as dst:
        dst.write(data_np, 1)
    print(f"Saved clipped sediment thickness to {target_tif}")

    # Stats
    finite_mask = np.isfinite(data_np)
    if finite_mask.any():
        min_v = float(np.nanmin(data_np))
        max_v = float(np.nanmax(data_np))
        mean_v = float(np.nanmean(data_np))
        median_v = float(np.nanmedian(data_np))
    else:
        min_v = max_v = mean_v = median_v = float("nan")

    with rasterio.open(target_tif) as src:
        print("CRS:", src.crs)
        print("Bounds:", src.bounds)
        print("Resolution:", src.res)
        print("Units:", units)
    print(f"Statistics -> min: {min_v:.2f}, max: {max_v:.2f}, mean: {mean_v:.2f}, median: {median_v:.2f}")
    print("Expected range (paper): 1000-14400 m")

    # Visualization
    cmap = LinearSegmentedColormap.from_list("sedthick", ["#0b3c49", "#3d7ea6", "#a8d08d", "#d4a05c", "#8b5a2b"])
    extent = [float(lon_vals_clip.min()), float(lon_vals_clip.max()), float(lat_vals_clip.min()), float(lat_vals_clip.max())]
    plt.figure(figsize=(10, 8))
    plt.imshow(data_np, origin="upper", extent=extent, cmap=cmap)
    cbar = plt.colorbar()
    cbar.set_label(f"Sediment thickness ({units})")
    plt.title("Sediment Thickness - Morocco EEZ (m)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    target_png.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(target_png, dpi=300)
    plt.close()
    print(f"Saved map to {target_png}")

    print("Sediment thickness subset ready for analysis.")

dask not installed; opening without chunking (will load subset to memory).
Dataset summary (lazy):
<xarray.Dataset>
Dimensions:  (lon: 4321, lat: 2161)
Coordinates:
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.8 ... 179.8 179.9 180.0
  * lat      (lat) float64 -90.0 -89.92 -89.83 -89.75 ... 89.75 89.83 89.92 90.0
Data variables:
    z        (lat, lon) float32 ...
Attributes:
    Conventions:  COARDS, CF-1.5
    title:        Produced by grdmath
    history:      grdclip GlobSed_v2.nc -Sb0/0 -GGlobSed-v2.nc
    description:  
    GMT_version:  5.4.2 (r18461) [64-bit]
Variables: ['z']
Saved clipped sediment thickness to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\sediment_thickness_morocco.tif
CRS: EPSG:4326
Bounds: BoundingBox(left=-20.0, bottom=20.91666666666758, right=-0.9166666666710057, top=37.0)
Resolution: (0.08333333333331439, 0.0833333333333286)
Units: unknown
Statistics -> min: 1.00, max: 14415.00, mean: 3233.18, median: 1969.50
Expected range (paper):

## Dataset: Shipping Routes / Vessel Density (EMODnet 2020)

In [32]:
from pathlib import Path
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.transform import array_bounds, xy
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from shapely.geometry import box
from shapely.ops import transform as shp_transform
from pyproj import Transformer

# Paths
shipping_candidates = [
    RAW_DATA_DIR / "vesseldensity_all_2020.tif",
    RAW_DATA_DIR / "shipping_density_emodnet_2020.tif",
]
shipping_src = next((p for p in shipping_candidates if p.exists()), None)
clipped_tif = RAW_DATA_DIR / "shipping_density_morocco_2020.tif"
eez_path = RAW_DATA_DIR / "morocco_eez.shp"
map_png = OUTPUTS_DIR / "shipping_density_morocco_2020_map.png"
restriction_tif = PROCESSED_DATA_DIR / "shipping_routes_restriction_2020.tif"

if shipping_src is None:
    print("Vessel density raster not found. Place 'vesseldensity_all_2020.tif' or 'shipping_density_emodnet_2020.tif' in data/raw.")
elif not eez_path.exists():
    print(f"EEZ boundary not found: {eez_path}")
else:
    eez = gpd.read_file(eez_path)
    with rasterio.open(shipping_src) as src:
        print(f"Source: {shipping_src.name}")
        print("CRS:", src.crs)
        print("Bounds:", src.bounds)
        print("Resolution:", src.res)
        print("Dtype:", src.dtypes[0])
        print("NoData:", src.nodata)
        tags_units = src.tags().get("units", "vessel hours per km2 per year (assumed)")
        print("Units:", tags_units)

        data_full = src.read(1, masked=True)
        print(f"Full extent stats -> min: {data_full.min():.2f}, max: {data_full.max():.2f}, mean: {data_full.mean():.2f}")

        # Confirm Morocco region coverage
        region_4326 = box(-20.0, 21.0, -1.0, 37.0)
        if src.crs and src.crs.to_epsg() != 4326:
            transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
            region_src = shp_transform(lambda x, y: transformer.transform(x, y), region_4326)
        else:
            region_src = region_4326
        covers_region = box(*src.bounds).intersects(region_src)
        print("Covers Morocco region?", covers_region)

        # Reproject EEZ to raster CRS
        if eez.crs != src.crs:
            eez = eez.to_crs(src.crs)
        shapes = eez.geometry.values

        clipped, clipped_transform = mask(src, shapes, crop=True, nodata=src.nodata)
        clipped_data = clipped[0]
        meta = src.meta.copy()
        meta.update({
            "height": clipped_data.shape[0],
            "width": clipped_data.shape[1],
            "transform": clipped_transform,
        })
        clipped_tif.parent.mkdir(parents=True, exist_ok=True)
        with rasterio.open(clipped_tif, "w", **meta) as dst:
            dst.write(clipped_data, 1)
        print(f"Saved clipped raster to {clipped_tif}")

    # Stats on clipped data
    nodata_val = meta.get("nodata", None)
    valid_mask = np.isfinite(clipped_data)
    if nodata_val is not None:
        valid_mask &= clipped_data != nodata_val
    if valid_mask.any():
        stats_min = float(np.nanmin(clipped_data[valid_mask]))
        stats_max = float(np.nanmax(clipped_data[valid_mask]))
        stats_mean = float(np.nanmean(clipped_data[valid_mask]))
        stats_median = float(np.nanmedian(clipped_data[valid_mask]))
    else:
        stats_min = stats_max = stats_mean = stats_median = float("nan")

    zero_count = int(np.count_nonzero(valid_mask & (clipped_data == 0)))
    traffic_count = int(np.count_nonzero(valid_mask & (clipped_data > 0)))
    total_count = int(np.count_nonzero(valid_mask))
    print(f"Clipped stats -> min: {stats_min:.2f}, max: {stats_max:.2f}, mean: {stats_mean:.2f}, median: {stats_median:.2f}")
    print(f"Pixel counts -> zero: {zero_count}, traffic (>0): {traffic_count}, total valid: {total_count}")

    # Max location
    if traffic_count > 0:
        masked_vals = np.ma.array(clipped_data, mask=~valid_mask)
        max_pos = np.unravel_index(masked_vals.argmax(), masked_vals.shape)
        max_val = float(masked_vals.max())
        max_lon, max_lat = xy(clipped_transform, max_pos[0], max_pos[1], offset="center")
        print(f"Max vessel density: {max_val:.2f} at ({max_lat:.4f}N, {max_lon:.4f}E)")
    else:
        max_val = float("nan")
        max_lon = max_lat = float("nan")

    # Restriction mask (vessel density > 0)
    restriction = np.where(valid_mask & (clipped_data > 0), 1, 0).astype("uint8")
    restr_meta = meta.copy()
    restr_meta.update({"dtype": "uint8", "nodata": 0, "compress": "lzw"})
    PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)
    with rasterio.open(restriction_tif, "w", **restr_meta) as dst:
        dst.write(restriction, 1)
    print(f"Saved restriction mask to {restriction_tif}")

    # Area percentages
    eez_area_km2 = float(eez.to_crs("EPSG:6933").area.sum() / 1e6)
    frac_restricted = (traffic_count / total_count) if total_count else 0.0
    area_restricted_km2 = frac_restricted * eez_area_km2
    area_open_km2 = eez_area_km2 - area_restricted_km2
    print(f"EEZ area (km2): {eez_area_km2:.2f}")
    print(f"Restricted (>0 traffic): {area_restricted_km2:.2f} km2 ({frac_restricted*100:.2f}%)")
    print(f"Open (zero traffic or nodata): {area_open_km2:.2f} km2 ({(1-frac_restricted)*100:.2f}%)")

    # Visualization
    plot_data = np.ma.masked_where(~valid_mask, clipped_data)
    vmax = float(np.nanmax(plot_data)) if traffic_count > 0 else 1.0
    bounds = [0, 1, 100, 1000, max(vmax, 1000) + 1]
    colors = ["#08306b", "#1c9099", "#41b6c4", "#fdae61", "#e31a1c"]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bounds, len(colors))
    cmap.set_bad("#f0f0f0")

    height, width = plot_data.shape
    left, bottom, right, top = array_bounds(height, width, clipped_transform)
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(plot_data, origin="upper", extent=[left, right, bottom, top], cmap=cmap, norm=norm)
    eez.boundary.plot(ax=ax, edgecolor="black", linewidth=0.8)
    cbar = plt.colorbar(im, ax=ax, fraction=0.035, pad=0.02)
    cbar.set_label("Vessel hours per km2 per year")
    ax.set_title("Vessel Density - Morocco EEZ (2020)")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    # Simple scale bar (approximate)
    def add_scale_bar(ax, length_km=100):
        if eez.crs and eez.crs.is_projected:
            length_units = length_km * 1000
        else:
            length_units = length_km / 111.0
        x_span = right - left
        y_span = top - bottom
        x_start = left + 0.05 * x_span
        y_start = bottom + 0.05 * y_span
        ax.plot([x_start, x_start + length_units], [y_start, y_start], color="black", linewidth=2)
        ax.text(x_start + length_units / 2, y_start + 0.01 * y_span, f"{length_km} km", ha="center", va="bottom", fontsize=9)
    add_scale_bar(ax, length_km=100)

    # North arrow
    ax.annotate("N", xy=(0.95, 0.1), xytext=(0.95, 0.25), xycoords="axes fraction", arrowprops=dict(facecolor="black", width=2, headwidth=8), ha="center", va="center")

    map_png.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(map_png, dpi=300)
    plt.close()
    print(f"Saved vessel density map to {map_png}")

    print("Vessel density processing complete.")

Source: vesseldensity_all_2020.tif
CRS: EPSG:3035
Bounds: BoundingBox(left=-623000.0, bottom=604000.0, right=6885000.0, top=7035000.0)
Resolution: (1000.0, 1000.0)
Dtype: float32
NoData: -3.4028234663852886e+38
Units: vessel hours per km2 per year (assumed)
Full extent stats -> min: 0.00, max: 49667.04, mean: 0.95
Covers Morocco region? True
Saved clipped raster to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\shipping_density_morocco_2020.tif
Clipped stats -> min: 0.00, max: 5399.58, mean: 0.40, median: 0.05
Pixel counts -> zero: 13804, traffic (>0): 266605, total valid: 280409
Max vessel density: 5399.58 at (1567500.0000N, 2919500.0000E)
Saved restriction mask to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\processed\shipping_routes_restriction_2020.tif
EEZ area (km2): 280455.87
Restricted (>0 traffic): 266649.56 km2 (95.08%)
Open (zero traffic or nodata): 13806.31 km2 (4.92%)
Saved vessel density map to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\d

In [5]:
# Extract Morocco EEZ from downloaded world GPKG
source_gpkg = Path("C:/Users/NameLess/Desktop/DATA RAW/World_EEZ_v11_20191118_gpkg/eez_v11.gpkg")
output_gpkg = RAW_DATA_DIR / "morocco_eez.gpkg"
output_shp = RAW_DATA_DIR / "morocco_eez.shp"

if not source_gpkg.exists():
    print(f"Source EEZ file not found: {source_gpkg}")
else:
    print(f"Reading {source_gpkg}")
    world_eez = gpd.read_file(source_gpkg)

    if "ISO_TER1" in world_eez.columns:
        morocco = world_eez[world_eez["ISO_TER1"].str.upper() == "MAR"]
    elif "TERRITORY1" in world_eez.columns:
        morocco = world_eez[world_eez["TERRITORY1"].str.contains("Morocco", case=False, na=False)]
    else:
        morocco = world_eez.head(0)

    print(f"Found {len(morocco)} Morocco EEZ features")
    if morocco.empty:
        print("No Morocco EEZ features found; check column names.")
    else:
        morocco.to_file(output_gpkg, driver="GPKG")
        morocco.to_file(output_shp)
        print(f"Saved Morocco EEZ to: {output_gpkg}")
        print(f"Saved Morocco EEZ (shp) to: {output_shp}")

Reading C:\Users\NameLess\Desktop\DATA RAW\World_EEZ_v11_20191118_gpkg\eez_v11.gpkg
Found 1 Morocco EEZ features
Saved Morocco EEZ to: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_eez.gpkg
Saved Morocco EEZ (shp) to: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_eez.shp


## Dataset: Protected Areas (WDPA Jan 2026, Morocco)
- Source: UNEP-WCMC WDPA January 2026 polygons (MAR subset).
- This workflow uses the updated 2024/2026 release (more conservative) versus the paper's WDPA Oct 2020.
- Steps: load WDPA, filter marine (`MARINE` 1/2 or designation contains "marine"/"MPA"), clip to Morocco EEZ, save marine subset, build 2000 m buffers (Table 3 requirement), visualize, compute stats, and rasterize a ~1 km restriction mask.

In [34]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

wdpa_path = RAW_DATA_DIR / "WDPA_WDOECM_Jan2026_Public_MAR_shp-polygons.shp"
eez_path = RAW_DATA_DIR / "morocco_eez.shp"
shoreline_path = RAW_DATA_DIR / "shoreline_morocco.shp"
marine_out = RAW_DATA_DIR / "protected_areas_morocco_marine.shp"
buffer_out = PROCESSED_DATA_DIR / "protected_areas_buffer_2000m.shp"
map_png = OUTPUTS_DIR / "protected_areas_morocco_2026_map.png"
restriction_tif = PROCESSED_DATA_DIR / "protected_areas_restriction.tif"
BUFFER_METERS = 2000
RASTER_RES_DEG = 0.01
BOUNDS = (-20.0, 21.0, -1.0, 37.0)  # west, south, east, north

if not wdpa_path.exists():
    print(f"WDPA shapefile not found at {wdpa_path}")
elif not eez_path.exists():
    print(f"EEZ boundary not found at {eez_path} (run EEZ extraction cell).")
else:
    wdpa = gpd.read_file(wdpa_path)
    print("WDPA CRS:", wdpa.crs)
    print("WDPA feature count:", len(wdpa))
    print("Columns:", list(wdpa.columns))
    print("First 5 rows:")
    print(wdpa.head())

    marine_col = wdpa.get("MARINE")
    desig_col = wdpa.get("DESIG_ENG")
    name_col = wdpa.get("NAME")
    if marine_col is None:
        marine_mask = pd.Series([False] * len(wdpa))
    else:
        marine_mask = marine_col.astype(str).str.strip().isin(["1", "2"])
    if desig_col is not None:
        marine_mask |= desig_col.str.contains("marine|mpa", case=False, na=False)
    marine = wdpa[marine_mask].copy()

    if marine.empty:
        print("No marine or partly marine protected areas detected.")
    else:
        print(f"Marine/partly marine features: {len(marine)}")

        # Clip to EEZ
        eez = gpd.read_file(eez_path)
        if eez.crs != marine.crs:
            eez = eez.to_crs(marine.crs)
        marine_clipped = gpd.overlay(marine, eez, how="intersection")
        marine_clipped = marine_clipped[~marine_clipped.geometry.is_empty].copy()

        # Persist marine subset within EEZ
        marine_out.parent.mkdir(parents=True, exist_ok=True)
        marine_clipped.to_file(marine_out)
        print(f"Saved marine subset to {marine_out}")

        # Stats
        rep_area = pd.to_numeric(marine_clipped.get("REP_AREA", pd.Series([], dtype=float)), errors="coerce")
        rep_area_sum = float(rep_area.fillna(0).sum()) if not rep_area.empty else float("nan")
        marine_area_km2 = float(marine_clipped.to_crs("EPSG:6933").area.sum() / 1e6)
        name_series = marine_clipped.get("NAME", pd.Series([], dtype=str)).dropna()
        names = sorted(name_series.unique()) if not name_series.empty else []
        desig_counts = marine_clipped.get("DESIG_ENG", pd.Series([], dtype=str)).fillna("Unknown").value_counts()

        print(f"Marine protected areas in EEZ: {len(marine_clipped)}")
        print(f"Total REP_AREA (km2): {rep_area_sum:.2f}")
        print(f"Clipped marine area (km2): {marine_area_km2:.2f}")
        print("Names:", names)
        print("Designation counts:")
        print(desig_counts.to_string())

        # 2000 m buffer per Table 3 restriction
        marine_proj = marine_clipped.to_crs("EPSG:6933")
        buffered_geom = marine_proj.buffer(BUFFER_METERS)
        buffer_union = buffered_geom.unary_union
        buffer_gdf = gpd.GeoDataFrame({"NAME": ["buffer_2000m"]}, geometry=[buffer_union], crs="EPSG:6933")
        buffer_area_km2 = float(buffer_gdf.area.sum() / 1e6)
        buffer_latlon = buffer_gdf.to_crs("EPSG:4326")
        buffer_out.parent.mkdir(parents=True, exist_ok=True)
        buffer_latlon.to_file(buffer_out)
        print(f"Saved 2000 m buffer to {buffer_out}")

        # EEZ area and coverage
        eez_area_km2 = float(eez.to_crs("EPSG:6933").area.sum() / 1e6)
        pct_buffer = (buffer_area_km2 / eez_area_km2 * 100) if eez_area_km2 else float("nan")
        print(f"EEZ area (km2): {eez_area_km2:.2f}")
        print(f"Buffer area (km2): {buffer_area_km2:.2f} ({pct_buffer:.3f}% of EEZ)")

        # Optional coastline overlay (skip if sidecar files missing)
        coastline = None
        if shoreline_path.exists():
            shx_path = shoreline_path.with_suffix(".shx")
            if not shx_path.exists():
                print(f"Coastline shapefile missing .shx -> skipping coastline overlay ({shoreline_path})")
            else:
                try:
                    coastline = gpd.read_file(shoreline_path)
                except Exception as exc:
                    print(f"Could not read coastline shapefile ({shoreline_path}): {exc}")
                    coastline = None
            if coastline is not None:
                if coastline.crs is None:
                    coastline.set_crs("EPSG:4326", inplace=True)
                elif coastline.crs.to_epsg() != 4326:
                    coastline = coastline.to_crs("EPSG:4326")
        else:
            print(f"Coastline file not found: {shoreline_path}")

        # Plot
        fig, ax = plt.subplots(figsize=(10, 8))
        buffer_latlon.plot(ax=ax, color="#f6b2b2", edgecolor="#d73027", linewidth=0.8, alpha=0.35, label="2000 m buffer")
        marine_plot = marine_clipped.to_crs("EPSG:4326")
        marine_plot.plot(ax=ax, column="DESIG_ENG", cmap="tab20", legend=True, legend_kwds={"title": "Designation", "loc": "upper right", "bbox_to_anchor": (1.15, 1)})
        designation_legend = ax.get_legend()
        marine_plot.boundary.plot(ax=ax, color="green", linewidth=0.8)
        eez.to_crs("EPSG:4326").boundary.plot(ax=ax, color="black", linewidth=1.0, label="Morocco EEZ")
        if coastline is not None:
            coastline.plot(ax=ax, color="none", edgecolor="gray", linewidth=0.5, label="Coastline")

        ax.set_title("Marine Protected Areas - Morocco EEZ (WDPA Jan 2026)")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")

        # Secondary legend for buffers and EEZ
        custom_handles = [
            Patch(facecolor="#f6b2b2", edgecolor="#d73027", alpha=0.35, label="2000 m buffer"),
            Line2D([0], [0], color="black", lw=1.0, label="EEZ boundary"),
        ]
        if coastline is not None:
            custom_handles.append(Line2D([0], [0], color="gray", lw=0.8, label="Coastline"))
        ax.legend(handles=custom_handles, loc="lower left", fontsize=9)
        if designation_legend is not None:
            ax.add_artist(designation_legend)

        map_png.parent.mkdir(parents=True, exist_ok=True)
        plt.tight_layout()
        plt.savefig(map_png, dpi=300)
        plt.close()
        print(f"Saved map to {map_png}")

        # Locations (centroid lat/lon)
        marine_centroids = marine_plot.copy()
        marine_centroids["centroid_lon"] = marine_centroids.geometry.centroid.x
        marine_centroids["centroid_lat"] = marine_centroids.geometry.centroid.y
        print("Centroid locations (lat, lon):")
        for _, row in marine_centroids.iterrows():
            nm = row.get("NAME", "Unnamed")
            print(f" - {nm}: {row['centroid_lat']:.4f}, {row['centroid_lon']:.4f}")

        # Restriction raster (~1 km at 0.01 degree)
        west, south, east, north = BOUNDS
        width = int(np.ceil((east - west) / RASTER_RES_DEG))
        height = int(np.ceil((north - south) / RASTER_RES_DEG))
        transform = from_origin(west, north, RASTER_RES_DEG, RASTER_RES_DEG)
        buffer_shapes = [(geom, 1) for geom in buffer_latlon.geometry if geom and not geom.is_empty]
        if not buffer_shapes:
            print("No buffer geometries to rasterize.")
        else:
            raster = rasterize(buffer_shapes, out_shape=(height, width), transform=transform, fill=0, dtype="uint8")
            restriction_tif.parent.mkdir(parents=True, exist_ok=True)
            profile = {
                "driver": "GTiff",
                "height": height,
                "width": width,
                "count": 1,
                "dtype": "uint8",
                "crs": "EPSG:4326",
                "transform": transform,
                "compress": "lzw",
                "nodata": 0,
            }
            with rasterio.open(restriction_tif, "w", **profile) as dst:
                dst.write(raster, 1)
            print(f"Saved restriction raster to {restriction_tif}")

        print("Marine protected areas processing complete.")

WDPA CRS: EPSG:4326
WDPA feature count: 144
Columns: ['SITE_ID', 'SITE_PID', 'SITE_TYPE', 'NAME_ENG', 'NAME', 'DESIG', 'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'REALM', 'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE', 'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'GOVSUBTYPE', 'OWN_TYPE', 'OWNSUBTYPE', 'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'PRNT_ISO3', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'INLND_WTRS', 'OECM_ASMT', 'geometry']
First 5 rows:
     SITE_ID   SITE_PID SITE_TYPE              NAME_ENG                NAME  \
0  555570055  555570055      OECM               Rissani             Rissani   
1  555570056  555570056      OECM                Bouija              Bouija   
2  555570057  555570057      OECM                Outeta              Outeta   
3  555570058  555570058      OECM  Aferdou - Errachidia  Aferdou-Errachidia   
4  555570059  555570059      OECM           Ras Beriakh         Ras Beriakh   

                          DESIG               

C:\Users\NameLess\AppData\Local\Temp\ipykernel_2888\795921229.py:149: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  marine_centroids["centroid_lon"] = marine_centroids.geometry.centroid.x
C:\Users\NameLess\AppData\Local\Temp\ipykernel_2888\795921229.py:150: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  marine_centroids["centroid_lat"] = marine_centroids.geometry.centroid.y


## Dataset 3: EEZ Boundaries (Morocco extract)

## Dataset 7: Airports (Geofabrik OSM shapefile, no compiler needed)

In [10]:
# Extract airports from Geofabrik shapefile (no pyrosm build requirements)
from pathlib import Path
import zipfile
import io
import requests
import geopandas as gpd
import pandas as pd

zip_path = RAW_DATA_DIR / "morocco-latest-free.shp.zip"
extract_dir = RAW_DATA_DIR / "morocco_geofabrik"
output_path = RAW_DATA_DIR / "airports_morocco.geojson"
bbox = (-20.0, 21.0, -1.0, 36.0)  # west, south, east, north
url = "https://download.geofabrik.de/africa/morocco-latest-free.shp.zip"

try:
    if not zip_path.exists():
        print("Downloading Geofabrik Morocco shapefile ...")
        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded to {zip_path}")
    else:
        print("Geofabrik zip already present.")

    if not extract_dir.exists():
        extract_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)
        print(f"Extracted to {extract_dir}")
    else:
        print("Geofabrik data already extracted.")

    candidates = [
        extract_dir / "gis_osm_transport_p_free_1.shp",  # points
        extract_dir / "gis_osm_transport_a_free_1.shp",  # areas
    ]

    frames = []
    for shp in candidates:
        if not shp.exists():
            print(f"Missing expected file: {shp}")
            continue
        gdf = gpd.read_file(shp)
        # Prefer aeroway attribute; fallback to fclass
        if "aeroway" in gdf.columns:
            mask = gdf["aeroway"].isin(["airport", "aerodrome"])
        else:
            mask = gdf.get("fclass", pd.Series([], dtype=str)).isin(["airport", "aerodrome"])
        gdf = gdf[mask]
        if gdf.empty:
            continue
        if gdf.crs is None:
            gdf.set_crs("EPSG:4326", inplace=True)
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs("EPSG:4326")
        gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
        frames.append(gdf)

    if not frames:
        print("No airport features found; check downloaded Geofabrik files.")
    else:
        airports = pd.concat(frames, ignore_index=True)
        airports = airports[~airports.geometry.is_empty]
        airports.to_file(output_path, driver="GeoJSON")
        print(f"Saved {len(airports)} airports to {output_path}")
        print("Geometry types:", airports.geometry.geom_type.unique())
        coords = []
        for geom in airports.geometry:
            pt = geom if geom.geom_type == "Point" else geom.representative_point()
            coords.append((pt.y, pt.x))
        print("Airport coordinates (lat, lon):")
        for idx, (lat, lon) in enumerate(coords, 1):
            print(f"{idx}: {lat:.6f}, {lon:.6f}")
except Exception as exc:
    print(f"Error building airports layer: {exc}")

Downloaded to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco-latest-free.shp.zip
Extracted to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik
Missing expected file: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik\gis_osm_transport_p_free_1.shp
Saved 45 airports to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\airports_morocco.geojson
Geometry types: ['Polygon']
Airport coordinates (lat, lon):
1: 30.328394, -9.409119
2: 31.605873, -8.032457
3: 33.369284, -7.587009
4: 33.653636, -7.217889
5: 31.947233, -4.401284
6: 30.266018, -5.859218
7: 35.729194, -5.912969
8: 29.555724, -8.012098
9: 21.391020, -16.956787
10: 33.593647, -7.463048
11: 30.381860, -9.549720
12: 32.138682, -7.912236
13: 26.732453, -11.684223
14: 27.418294, -9.171708
15: 34.231915, -3.944656
16: 33.868063, -5.503121
17: 30.938197, -6.908926
18: 30.501754, -8.826298
19: 29.014195, -8.839525
20: 31.098702, -5.192406
21: 30.37615

## Dataset: Power Grid (Geofabrik OSM powerlines)

In [ ]:
# Build power grid layer from Geofabrik powerlines (skip if authoritative file already present)
from pathlib import Path
import zipfile
import shutil
import requests
import geopandas as gpd
import pandas as pd

zip_path = RAW_DATA_DIR / "morocco-latest-free.shp.zip"
extract_dir = RAW_DATA_DIR / "morocco_geofabrik"
fallback_dir = RAW_DATA_DIR / "morocco_geofabrik_power"
output_path = RAW_DATA_DIR / "power_grid_morocco.geojson"
bbox = (-20.0, 21.0, -1.0, 36.0)
url = "https://download.geofabrik.de/africa/morocco-latest-free.shp.zip"

try:
    if output_path.exists():
        print(f"Using existing power grid file: {output_path}")
    else:
        # Ensure zip present
        if not zip_path.exists():
            print("Downloading Geofabrik Morocco shapefile ...")
            with requests.get(url, stream=True, timeout=120) as r:
                r.raise_for_status()
                with open(zip_path, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Downloaded to {zip_path}")
        else:
            print("Geofabrik zip already present.")

        # Ensure extraction; if power files missing, extract to a clean fallback_dir
        def extract_into(target_dir: Path):
            if target_dir.exists():
                shutil.rmtree(target_dir)
            target_dir.mkdir(parents=True, exist_ok=True)
            with zipfile.ZipFile(zip_path, "r") as zf:
                zf.extractall(target_dir)
            print(f"Extracted to {target_dir}")

        # First use existing extraction if present
        if extract_dir.exists():
            print("Geofabrik data already extracted.")
        else:
            extract_into(extract_dir)

        def locate_sources(base_dir: Path):
            cand = [
                base_dir / "gis_osm_powerlines_free_1.shp",
                base_dir / "gis_osm_power_a_free_1.shp",
            ]
            missing = [p for p in cand if not p.exists()]
            return cand, missing

        sources, missing = locate_sources(extract_dir)
        if missing:
            print("Powerline files missing in existing extraction; re-extracting fresh...")
            extract_into(fallback_dir)
            sources, missing = locate_sources(fallback_dir)

        if missing:
            print("Still missing powerline layers after re-extract. Geofabrik package may have changed.")
            print("Files not found:")
            for p in missing:
                print(f" - {p}")

        frames = []
        for shp in sources:
            if not shp.exists():
                continue
            gdf = gpd.read_file(shp)
            if gdf.crs is None:
                gdf.set_crs("EPSG:4326", inplace=True)
            elif gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs("EPSG:4326")
            gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
            frames.append(gdf)

        if not frames:
            print("No powerline features found; check Geofabrik files.")
        else:
            power = gpd.GeoDataFrame(pd.concat(frames, ignore_index=True), crs="EPSG:4326")
            power = power[~power.geometry.is_empty]
            power.to_file(output_path, driver="GeoJSON")
            print(f"Saved {len(power)} powerline features to {output_path}")
            print("Geometry types:", power.geometry.geom_type.unique())
except Exception as exc:
    print(f"Error building power grid layer: {exc}")

Geofabrik zip already present.
Geofabrik data already extracted.
Powerline files missing in existing extraction; re-extracting fresh...
Extracted to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik_power
Still missing powerline layers after re-extract. Geofabrik package may have changed.
Files not found:
 - C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik_power\gis_osm_powerlines_free_1.shp
 - C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik_power\gis_osm_power_a_free_1.shp
No powerline features found; check Geofabrik files.


## Dataset: Ports (Geofabrik OSM transport)

In [12]:
# Build ports layer from Geofabrik transport features
from pathlib import Path
import zipfile
import requests
import geopandas as gpd
import pandas as pd

zip_path = RAW_DATA_DIR / "morocco-latest-free.shp.zip"
extract_dir = RAW_DATA_DIR / "morocco_geofabrik"
output_path = RAW_DATA_DIR / "ports_morocco.geojson"
bbox = (-20.0, 21.0, -1.0, 36.0)
url = "https://download.geofabrik.de/africa/morocco-latest-free.shp.zip"
port_classes = {"harbour", "port", "ferry_terminal"}

try:
    if not zip_path.exists():
        print("Downloading Geofabrik Morocco shapefile ...")
        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded to {zip_path}")
    else:
        print("Geofabrik zip already present.")

    if not extract_dir.exists():
        extract_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)
        print(f"Extracted to {extract_dir}")
    else:
        print("Geofabrik data already extracted.")

    sources = [
        extract_dir / "gis_osm_transport_p_free_1.shp",
        extract_dir / "gis_osm_transport_a_free_1.shp",
    ]

    frames = []
    for shp in sources:
        if not shp.exists():
            print(f"Missing expected file: {shp}")
            continue
        gdf = gpd.read_file(shp)
        mask = gdf.get("fclass", pd.Series([], dtype=str)).isin(port_classes)
        gdf = gdf[mask]
        if gdf.empty:
            continue
        if gdf.crs is None:
            gdf.set_crs("EPSG:4326", inplace=True)
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs("EPSG:4326")
        gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
        frames.append(gdf)

    if not frames:
        print("No port features found; check Geofabrik files.")
    else:
        ports = pd.concat(frames, ignore_index=True)
        ports = ports[~ports.geometry.is_empty]
        ports.to_file(output_path, driver="GeoJSON")
        print(f"Saved {len(ports)} port features to {output_path}")
        print("Geometry types:", ports.geometry.geom_type.unique())
except Exception as exc:
    print(f"Error building ports layer: {exc}")

Geofabrik zip already present.
Geofabrik data already extracted.
Missing expected file: C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\morocco_geofabrik\gis_osm_transport_p_free_1.shp
Saved 5 port features to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\ports_morocco.geojson
Geometry types: ['Polygon']


## Dataset: Tourism POIs (Geofabrik OSM POIs)

In [13]:
# Build tourism POIs layer from Geofabrik POIs
from pathlib import Path
import zipfile
import requests
import geopandas as gpd
import pandas as pd

zip_path = RAW_DATA_DIR / "morocco-latest-free.shp.zip"
extract_dir = RAW_DATA_DIR / "morocco_geofabrik"
output_path = RAW_DATA_DIR / "tourism_morocco.geojson"
bbox = (-20.0, 21.0, -1.0, 36.0)
url = "https://download.geofabrik.de/africa/morocco-latest-free.shp.zip"
# Broad set of tourism-related classes
fclass_keep = {
    "tourist_info", "information", "viewpoint", "museum", "gallery", "artwork",
    "attraction", "theme_park", "zoo", "aquarium", "hotel", "motel", "hostel",
    "guesthouse", "apartment", "chalet", "camp_site", "caravan_site", "picnic_site",
    "alpine_hut", "wilderness_hut", "resort"
}

try:
    if not zip_path.exists():
        print("Downloading Geofabrik Morocco shapefile ...")
        with requests.get(url, stream=True, timeout=120) as r:
            r.raise_for_status()
            with open(zip_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded to {zip_path}")
    else:
        print("Geofabrik zip already present.")

    if not extract_dir.exists():
        extract_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_dir)
        print(f"Extracted to {extract_dir}")
    else:
        print("Geofabrik data already extracted.")

    sources = [
        extract_dir / "gis_osm_pois_free_1.shp",
        extract_dir / "gis_osm_pois_a_free_1.shp",
    ]

    frames = []
    for shp in sources:
        if not shp.exists():
            print(f"Missing expected file: {shp}")
            continue
        gdf = gpd.read_file(shp)
        if "tourism" in gdf.columns:
            mask = gdf["tourism"].notna()
        else:
            mask = gdf.get("fclass", pd.Series([], dtype=str)).isin(fclass_keep)
        gdf = gdf[mask]
        if gdf.empty:
            continue
        if gdf.crs is None:
            gdf.set_crs("EPSG:4326", inplace=True)
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs("EPSG:4326")
        gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
        frames.append(gdf)

    if not frames:
        print("No tourism POIs found; check Geofabrik files.")
    else:
        tourism = pd.concat(frames, ignore_index=True)
        tourism = tourism[~tourism.geometry.is_empty]
        tourism.to_file(output_path, driver="GeoJSON")
        print(f"Saved {len(tourism)} tourism POIs to {output_path}")
        print("Geometry types:", tourism.geometry.geom_type.unique())
except Exception as exc:
    print(f"Error building tourism layer: {exc}")

Geofabrik zip already present.
Geofabrik data already extracted.
Saved 7866 tourism POIs to C:\Users\NameLess\Desktop\morocco-offshore-wind-gis\data\raw\tourism_morocco.geojson
Geometry types: ['Point' 'Polygon' 'MultiPolygon']


## Dataset 9: Submarine Cables (auto-download)



This dataset can be fetched directly from the TeleGeography repository.

## Dataset 9: Submarine Cables (MAT → GeoJSON conversion)
- Converts the provided MATLAB data to GeoJSON and saves to `data/raw/submarine_cables.geojson`.
- Source file expected at `C:/Users/NameLess/Desktop/DATA RAW/SubmarineCables-main/PolyLattice.mat`.
- Leaves the TeleGeography auto-download cell below unchanged.

In [7]:
def download_submarine_cables():

    """Download submarine cable data from TeleGeography."""

    url = "https://raw.githubusercontent.com/telegeography/www.submarinecablemap.com/master/web/public/api/v3/cable/cable-geo.json"

    output_path = RAW_DATA_DIR / "submarine_cables.geojson"

    if output_path.exists():

        print("Submarine cables data already exists.")

        return

    print("Downloading submarine cables data...")

    try:

        response = requests.get(url, stream=True, timeout=60)

        response.raise_for_status()

        total_size = int(response.headers.get("content-length", 0))

        with open(output_path, "wb") as f, tqdm(total=total_size, unit="B", unit_scale=True, desc="Submarine cables") as pbar:

            for chunk in response.iter_content(chunk_size=8192):

                f.write(chunk)

                pbar.update(len(chunk))

        print(f"Downloaded to {output_path}")

    except Exception as exc:

        print(f"Error downloading submarine cables: {exc}")



# Run download

download_submarine_cables()

Error downloading submarine cables: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/telegeography/www.submarinecablemap.com/master/web/public/api/v3/cable/cable-geo.json


## Final Data Inventory Summary

In [ ]:
# Refresh status

final_status_df = check_data_status()



print("=" * 60)

print("FINAL DATA COLLECTION STATUS")

print("=" * 60)

print(final_status_df.to_string(index=False))



downloaded = (final_status_df["Status"] == "OK").sum()

total = len(final_status_df)

progress = (downloaded / total) * 100

print(f"\nCollection progress: {downloaded}/{total} ({progress:.1f}%)")



missing = final_status_df[final_status_df["Status"] != "OK"]

if missing.empty:

    print("All datasets collected. Proceed to preprocessing.")

else:

    print("Missing datasets:")

    for _, row in missing.iterrows():

        print(f"  - {row['Dataset']} ({row['Filename']})")



# Save inventory

inventory_path = RAW_DATA_DIR / "_data_inventory.csv"

final_status_df.to_csv(inventory_path, index=False)

print(f"Inventory saved to {inventory_path}")

## Manual Download Checklist



Copy and check off items as you download them:



- [ ] Wind Speed (Global Wind Atlas)

- [ ] Bathymetry (GEBCO 2020)

- [ ] Power Grid (World Bank / OSM fallback)

- [ ] Ports (ANP or Natural Earth)

- [ ] Shoreline (GSHHG)

- [ ] Tourism POIs (OpenStreetMap)

- [ ] Airports (OpenStreetMap)

- [ ] Sediment Thickness (NOAA)

- [x] Submarine Cables (auto-download)

- [ ] Shipping Routes (EMODnet)

- [ ] Protected Areas (UNEP-WCMC)

- [ ] EEZ Boundaries (Marine Regions)

- [ ] Blue Flag Beaches (Blue Flag Global)

- [ ] Bird Migration Routes (Movebank)

- [ ] Offshore Wind Potential (World Bank ESMAP)

## Next Steps



After collecting all datasets:

1. Run 02_data_preprocessing.ipynb to standardize CRS, clip to EEZ, and resample.

2. Proceed to criteria mapping, fuzzy AHP, suitability analysis, and visualization.